# 함수 정의

In [65]:

import json
import os
import re
import time
from googleapiclient.discovery import build
import isodate
import emoji
from kiwipiepy import Kiwi
from kiwipiepy.utils import Stopwords
from krwordrank.word import KRWordRank
from krwordrank.word import summarize_with_keywords
from collections import Counter
import nltk
from nltk.corpus import stopwords
from konlpy.tag import Okt

# 환경 변수 및 변수 설정
API_KEY = os.getenv("YOUTUBE_API_KEY")
youtube = build("youtube", "v3", developerKey=API_KEY)  

CATEGORIES = {
    "News & Politics": "25",
    'Music' : "10",
    'Sports' : "17",
    'Gaming' : "20",
    'Science & Technology': "28"
}
# JSON 데이터 저장 함수
def save_to_json(data, filename):
    # data 폴더가 존재하지 않으면 생성
    if not os.path.exists('data'):
        os.makedirs('data')
    
    # 파일 경로를 data 폴더 아래로 설정
    filepath = os.path.join('data', filename)
    
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
        
    print(f"save_to_json : 데이터 저장: {filepath}")
    
    # JSON 데이터 로드 함수
# JSON 파일을 읽어서 딕셔너리로 반환하는 함수
def load_json(filename):
    # 파일 경로를 data 폴더 아래로 설정
    filepath = os.path.join('data', filename)
    with open(filepath, 'r', encoding='utf-8') as file:
        return json.load(file)
# value가 문자열배열의 배열일 때 한글자와 빈 문자열을 제외하고 문자열로 만드는 함수
def merge_values(data):
    """
    key: value에서 value가 문자열 배열의 배열일 때,
    하나의 문자열로 변환하고 빈 문자열("") 및 한 글자 단어는 제외하는 함수.

    :param data: dict, key: list of lists (e.g., { "key1": [["문장1", "문장2"], ["문장3"]] })
    :return: dict, key: merged string (e.g., { "key1": "문장1 문장2 문장3" })
    """
    merged_data = merge_values
    for key, values in data.items():
        if isinstance(values, list):  # 값이 리스트인지 확인
            merged_sentence = " ".join(
                s for sublist in values for s in sublist if s.strip() and len(s.strip()) > 1
            )  # 빈 문자열 & 한 글자 제외 후 공백으로 연결
            merged_data[key] = merged_sentence  # key: 병합된 문자열 형태로 저장

    return merged_data
# title과 tags를 하나의 문장으로 합치는 함수
def merge_clean_title_tags(data):
    """
    딕셔너리에서 'title'과 'tags'를 하나의 문장으로 변환하는 함수.
    
    :param data: dict (유튜브 비디오 정보)
    :return: dict (title과 tags가 합쳐진 문장)
    """
    merged_data = {}
    for category, videos in data.items():
        merged_data[category] = []  # 카테고리별 리스트 생성
        for video in videos:
            title = video.get("title", "").strip()  # title이 없으면 빈 문자열
            tags = video.get("tags", [])  # tags가 없거나 None이면 빈 리스트로 처리
            merged = " ".join([title] + tags)  # title과 tags 결합
            cleaned_merged = clean_text(merged)  # 텍스트 전처리
            merged_data[category].append(cleaned_merged)
    
    return merged_data
# 텍스트 데이터를 전처리하는 함수
def clean_text(text):
    # HTML 태그 제거
    text = re.sub(r'<[^>]+>', '', text)
    # &quot 제거
    text = text.replace("&quot", "")
    text = text.replace("&lt", "")
    text = text.replace("&gt", "")
    # URL, 멘션, 해시태그 제거
    text = re.sub(r'http\S+|www\S+|@\S+|#\S+', '', text)
    # 한글, 영어, 숫자, 공백을 제외한 모든 문자 제거
    text = re.sub(r"[^가-힣a-zA-Z0-9\s]", "", text)
    # 앞뒤 공백 제거
    text = text.strip()
    # 중복 공백 제거
    text = re.sub(r'\s{2,}', ' ', text)
    # 모든 이모지 제거
    text = emoji.replace_emoji(text, replace="")
    # 영어를 소문자로 변환
    text = text.lower()
    text = re.sub(r"윤대통령|윤대통|윤통|윤", "윤석열", text)
    text = re.sub(r"장원", "홍장원", text)
    text = re.sub(r"계엄령", "계엄", text)

    return text
# YOUTUBE API를 통해 동영상 데이터 가져오기
def fetch_trending_videos(category_id, region_code="KR", max_results=200):
    
    videos = []
    next_page_token = None

    while len(videos) < max_results:
        try:
            request = youtube.videos().list(
                part="snippet,statistics,contentDetails",
                chart="mostPopular",
                regionCode=region_code,
                videoCategoryId=category_id,
                maxResults=min(50, max_results - len(videos)),
                pageToken=next_page_token
            )
            response = request.execute()

            for item in response.get("items", []):
                duration = isodate.parse_duration(item["contentDetails"]["duration"])
                duration_in_seconds = duration.total_seconds()  #초로 바꾸기기

                if duration_in_seconds > 80:  # 80초 이상의 동영상만 가져오기
                    videos.append({
                        "video_id": item["id"],
                        "title": item["snippet"]["title"],
                        "description": item["snippet"]["description"],
                        "tags": item["snippet"].get("tags", []),
                        "duration": str(duration),
                        "view_count": int(item["statistics"].get("viewCount", 0)),
                        "like_count": int(item["statistics"].get("likeCount", 0)),
                        "comment_count": int(item["statistics"].get("commentCount", 0)),
                        "category_id": category_id,
                    })

            next_page_token = response.get("nextPageToken")
            if not next_page_token:
                break

        except Exception as e:
            print(f"Error fetching videos: {e}")
            time.sleep(5)  # 잠시 대기 후 다시 시도

    return videos
# 모든 동영상 데이터 가져오기기
def write_all_videos(output_file="raw_video_data.json"):
    all_videos = {}

    for category_name, category_id in CATEGORIES.items():
        print(f"비디오 가져올 카테고리 : {category_name}")
        videos = fetch_trending_videos(category_id, region_code="KR", max_results=200)
        all_videos[category_name] = videos
        print(f"{len(videos)}개 완료.")
        
    save_to_json(all_videos, output_file)
    print(f"write_all_videos : 모든 비디오 데이터가 '{output_file}'에 저장되었습니다.")
# 비디오 댓글 가져오기 함수
def fetch_video_comments(video_id, max_results=100):
    comments = []
    next_page_token = None

    while len(comments) < max_results:
        try:
            request = youtube.commentThreads().list(
                part="snippet",
                videoId=video_id,
                maxResults=min(50, max_results - len(comments)),
                pageToken=next_page_token
            )
            response = request.execute()

            for item in response.get("items", []):
                comment = item["snippet"]["topLevelComment"]["snippet"]["textDisplay"]
                comments.append(comment)

            next_page_token = response.get("nextPageToken")
            if not next_page_token:
                if not comments:
                    print(f"{video_id} - 댓글 없음")
                break

        except Exception as e:
            break

    return comments
# 비디오 댓글 저장 함수
def write_video_comments(input_file, output_file):
    # JSON 데이터 로드
    data = load_json(input_file)
    
    # 비디오 댓글 가져오기 및 저장
    all_comments = {}
    for category, videos in data.items():
        all_comments[category] = []  # 카테고리별 리스트 생성

        for video in videos:
            video_id = video["video_id"]
            comments = fetch_video_comments(video_id, max_results=1000)
            all_comments[category].append({"video_id": video_id, "comments": comments})
             
    # 결과 저장
    save_to_json(all_comments, output_file)
    print(f"get_video_comments : 비디오 댓글이 '{output_file}'에 저장되었습니다.")
# 비디오 댓글 전처리 저장 함수
def write_clean_text(input_file, output_file):
    data = load_json(input_file)
    processed_data = {} 
    
    for category, videos in data.items():
        processed_data[category] = []  # 카테고리별 리스트 생성
        if isinstance(videos, list):  # 값이 리스트인지 확인
            for video in videos:
                video_id = video["video_id"]
                comments = video.get("comments", [])
                cleaned_comments = [clean_text(comment) for comment in comments]
                processed_data[category].append({"video_id": video_id, "comments": cleaned_comments})
        
    # 총 문장 수 계산
    total_sentences = sum(len(video["comments"]) for videos in processed_data.values() for video in videos)
    print(f"wirte_clean : 총 문장 수 = {total_sentences}")  
    save_to_json(processed_data, output_file)
# Kiwi 객체 생성 함수
def study_extract_kiwi(input_file, output_file, train_file):
    # Kiwi 객체 생성
    kiwi = Kiwi()
    # Load the cleaned video comments
    data = load_json(input_file)
    train_data = load_json(train_file)
    kiwi_objects = {}
    
    # for category, videos in data.items():
    #     if isinstance(videos, list):  # Check if the value is a list
    #         train_data[category] = []  # Initialize list for each category
    #         for video in videos:
    #             comments = video.get("comments", [])
    #             train_data[category].extend(comments)  # Add corrected comments to train_data
    # save_to_json(train_data, "train_data.json")
    # 훈련
    for category, comments in train_data.items():
        kiwi = Kiwi()
        kiwi.extract_add_words(comments, min_cnt=5, max_word_len=10, min_score=0.25, pos_score=0.5, lm_filter=True)
        kiwi_objects[category] = kiwi
        
    print("study_kiwi : 훈련 완료")
    print(kiwi_objects)
    
    write_tokenize_nouns(input_file, output_file, kiwi_objects)
    
    return kiwi_objects
# 형태소 분석 및 명사 추출 함수
def write_tokenize_nouns(input_file, output_file, kiwi_objects):
    
    kiwi_stopwords = Stopwords()
    
    eng_stopwords = list(stopwords.words('english'))  # 영어 불용어 사전
    
    for stopword in eng_stopwords:
        kiwi_stopwords.add((stopword, 'SL'))
    
    tokenized_nouns = {}
    data = load_json(input_file)

    for category, videos in data.items():
        tokenized_nouns[category] = []  # 카테고리별 리스트 생성

        for video in videos:
            video_id = video["video_id"]
            comments = video.get("comments", [])

            # 댓글을 형태소 분석 후 명사만 추출
            tokenized = []
            for comment in comments:
                tokens = kiwi_objects[category].tokenize(comment, stopwords = kiwi_stopwords )  # 카테고리별 Kiwi 객체 사용하여 형태소 분석
                nouns = [
                    token.form for token in tokens
                    if token.tag in ["NNP", "SL"]  # 명사(NNP)만 추출
                    and len(token.form) > 1  # 1글자 제외
                ]
                tokenized.append(nouns)

            # 구조 유지
            tokenized_nouns[category].append({"video_id": video_id, "nouns": tokenized})

    save_to_json(tokenized_nouns, output_file)    
# KR-WordRank 객체 생성 함수
def work_krRank(input_file, output_file):

    data = load_json(input_file)
    ranked_keywords = {}  # ✅ 모든 카테고리의 결과를 저장할 딕셔너리

    for category, values in data.items():
        print(f"🔍 카테고리: {category}")
        print(f"📌 데이터 타입: {type(values)}")

        if not isinstance(values, list) or not values:
            print(f"⚠️ '{category}' 카테고리에서 키워드 분석을 건너뜀 (데이터 없음)")
            continue

        # ✅ 키워드 추출 실행
        keywords = summarize_with_keywords(values)
        # ✅ 점수가 높은 순으로 정렬하여 상위 100개만 선택
        sorted_keywords = sorted(keywords.items(), key=lambda x: x[1], reverse=True)[:1000]
        ranked_keywords[category] = sorted_keywords  # ✅ 결과 저장

        # ✅ 현재 카테고리의 결과 출력
        print(f"✅ {category} 키워드 추출 완료")

    # ✅ 모든 카테고리의 결과를 한 번에 저장
    save_to_json(ranked_keywords, output_file)
    print(f"✅ 키워드 랭킹이 '{output_file}'에 저장되었습니다.")
# kr-wrodrank cleaned data
def work_krRank2(input_file):
    min_count = 10   # 단어의 최소 출현 빈도수
    max_length = 10  # 단어의 최대 길이
    wordrank_extractor = KRWordRank(min_count=min_count, max_length=max_length)

    data = load_json(input_file)
    
    beta = 0.85    # PageRank의 감쇄 계수
    max_iter = 10  # 반복 횟수
    ranked_keywords = {}  # ✅ 모든 카테고리의 결과를 저장할 딕셔너리
    noun_keywords = {}  # ✅ 명사 키워드만 저장할 딕셔너리
    for category, videos in data.items():
        for video in videos:
            comments = video.get("comments", [])
            document = " ".join(comments)  # ✅ 모든 댓글을 하나의 문서로 결합

        # ✅ 데이터 상태 확인
        print(f"✅ 카테고리: {category}, 댓글 개수: {len(comments)}")
        print(f"📌 문서 데이터 샘플: {comments[:10]}")  # 일부 출력

        # ✅ 키워드 추출 실행 (빈 리스트가 아님을 보장)
        try:
            keywords, rank, graph = wordrank_extractor.extract(comments, beta, max_iter)
        except Exception as e:
            print(f"❌ 오류 발생: {category} 카테고리에서 키워드 추출 실패. 오류 메시지: {e}")
            continue
        
        # 4️⃣ Kiwi를 사용해서 명사만 필터링
        kiwi = Kiwi()

        noun_keywords = {
            word: score for word, score in keywords.items()
            if any(token.tag.startswith("NNP") for token in kiwi.tokenize(word))
        }

        sorted_noun_keywords = sorted(noun_keywords.items(), key=lambda x: x[1], reverse=True)[:100]
        
        ranked_keywords[category] = sorted_noun_keywords  # ✅ 결과 저장

        # ✅ 현재 카테고리의 결과 출력
        print(f"Category: {category}")
        print(sorted_noun_keywords[:10])  # 상위 10개 키워드만 출력

    # ✅ 모든 카테고리의 결과를 한 번에 저장
    save_to_json(ranked_keywords, "ranked_keywords.json")
    print("✅ work_krRank2 : 키워드 랭킹이 'ranked_keywords.json'에 저장되었습니다.")
# 텍스트 merge
def merge_nouns(input_file, output_file):
    data = load_json(input_file)
    merged_data = {}  # 결과 저장할 딕셔너리

    for category, videos in data.items():
        category_texts = []  # 현재 카테고리의 video별 문자열 리스트

        for video in videos:
            nouns = video.get("nouns", [])  # nouns 가져오기
            
            # ✅ video의 모든 nouns를 하나의 문자열로 변환
            merged_text = " ".join([" ".join(noun_list) for noun_list in nouns])
            
            if merged_text:  # 비어있지 않은 경우 추가
                category_texts.append(merged_text)

        # ✅ 최종적으로 category별 str 리스트 저장
        category_texts = [re.sub(r'\s{2,}', ' ', text) for text in category_texts]
        merged_data[category] = category_texts

    save_to_json(merged_data, output_file)
    print("✅ merge_nouns : 명사 데이터가 'merged_nouns.json'에 저장되었습니다.")
     
def merge_comments(input_file, output_file):
    data = load_json(input_file)
    merged_data = {}  # 결과 저장할 딕셔너리

    for category, videos in data.items():
        category_comments = []  # 현재 카테고리의 video별 문자열 리스트

        for video in videos:
            comments = video.get("comments", [])  # nouns 가져오기
                
                # ✅ video의 모든 comments를 하나의 문자열로 변환
            merged_comments = " ".join(comments)  
            if merged_comments:  # 비어있지 않은 경우 추가
                category_comments.append(merged_comments)

            # ✅ 최종적으로 category별 str 리스트 저장
            category_comments = [re.sub(r'\s{2,}', ' ', text) for text in category_comments]
            merged_data[category] = category_comments

    save_to_json(merged_data, output_file)
    print("✅ merge_nouns : 명사 데이터가 'merged_nouns.json'에 저장되었습니다.")  

In [2]:
write_all_videos("raw_video_data.json")


비디오 가져올 카테고리 : News & Politics
38개 완료.
비디오 가져올 카테고리 : Music
30개 완료.
비디오 가져올 카테고리 : Sports
9개 완료.
비디오 가져올 카테고리 : Gaming
71개 완료.
비디오 가져올 카테고리 : Science & Technology
67개 완료.
save_to_json : 데이터 저장: data\raw_video_data.json
write_all_videos : 모든 비디오 데이터가 'raw_video_data.json'에 저장되었습니다.


In [3]:
write_video_comments("raw_video_data.json", "video_comments.json")

fgorh0Oh4pM - 댓글 없음
save_to_json : 데이터 저장: data\video_comments.json
get_video_comments : 비디오 댓글이 'video_comments.json'에 저장되었습니다.


In [62]:
write_clean_text("video_comments.json", "cleaned_video_comments.json")

wirte_clean : 총 문장 수 = 79528
save_to_json : 데이터 저장: data\cleaned_video_comments.json


In [63]:
kiwi_objects = study_extract_kiwi("cleaned_video_comments.json","tokenized_nouns.json", "merged_comments.json")

study_kiwi : 훈련 완료
{'News & Politics': Kiwi(num_workers=1, model_path=None, integrate_allomorph=True, load_default_dict=True, load_typo_dict=True, model_type='knlm', typos=None, typo_cost_threshold=2.5), 'Music': Kiwi(num_workers=1, model_path=None, integrate_allomorph=True, load_default_dict=True, load_typo_dict=True, model_type='knlm', typos=None, typo_cost_threshold=2.5), 'Sports': Kiwi(num_workers=1, model_path=None, integrate_allomorph=True, load_default_dict=True, load_typo_dict=True, model_type='knlm', typos=None, typo_cost_threshold=2.5), 'Gaming': Kiwi(num_workers=1, model_path=None, integrate_allomorph=True, load_default_dict=True, load_typo_dict=True, model_type='knlm', typos=None, typo_cost_threshold=2.5), 'Science & Technology': Kiwi(num_workers=1, model_path=None, integrate_allomorph=True, load_default_dict=True, load_typo_dict=True, model_type='knlm', typos=None, typo_cost_threshold=2.5)}
save_to_json : 데이터 저장: data\tokenized_nouns.json


In [64]:
merge_nouns("tokenized_nouns.json", "merged_data.json")

save_to_json : 데이터 저장: data\merged_data.json
✅ merge_nouns : 명사 데이터가 'merged_nouns.json'에 저장되었습니다.


In [56]:
work_krRank("merged_data.json", "ranked_keywords.json")

🔍 카테고리: News & Politics
📌 데이터 타입: <class 'list'>
✅ News & Politics 키워드 추출 완료
🔍 카테고리: Music
📌 데이터 타입: <class 'list'>
✅ Music 키워드 추출 완료
🔍 카테고리: Sports
📌 데이터 타입: <class 'list'>
✅ Sports 키워드 추출 완료
🔍 카테고리: Gaming
📌 데이터 타입: <class 'list'>
✅ Gaming 키워드 추출 완료
🔍 카테고리: Science & Technology
📌 데이터 타입: <class 'list'>
✅ Science & Technology 키워드 추출 완료
save_to_json : 데이터 저장: data\ranked_keywords.json
✅ 키워드 랭킹이 'ranked_keywords.json'에 저장되었습니다.


In [8]:
merged_data = merge_clean_title_tags(load_json("raw_video_data.json"))

save_to_json(merged_data, "title_tags.json")

save_to_json : 데이터 저장: data\title_tags.json


In [19]:
merge_comments("cleaned_video_comments.json", "merged_comments.json")

save_to_json : 데이터 저장: data\merged_comments.json
✅ merge_nouns : 명사 데이터가 'merged_nouns.json'에 저장되었습니다.


In [ ]:
# Initialize Okt tokenizer
okt = Okt()

# Function to extract nouns using Okt


# Example usage
data = load_json('title_tags.json')

for category, list in data.items():
    for item in list:
        nouns = okt.nouns(item)
        print(nouns)
nouns = extract_nouns(sample_text)
print(nouns)